In [1]:
import pandas as pd
import os, glob
from owlready2 import *

import owlready2
print(owlready2.VERSION) ## Bugs when > 0.40 but works file at owlready2==0.40 

#```Merging ontologies:
#https://stackoverflow.com/questions/75102102/merge-two-ontologies-using-owlready
#```

from datetime import datetime

def NOW():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    return "Current Time = "+ str(current_time)

NOW()

def debug(onto):
    print("\n# Overview\n")
    for k in onto.classes():
        if len(k.instances()):
            print("* ","_"+str(k).split(".")[-1]+"_"," -- ",len(k.instances()), "instances.")

0.45


# Removing PROBONO Blueprints

In [2]:
ontoB = get_ontology("../kg/new/pbn_t3_5.owl").load()

ontoB.metadata.comment[4] = 'Version: 0.3.a'
ontoB.metadata.comment[5] = 'VersionComment: 0.3 - limited to PROBONO blueprints'

for k in ontoB.classes():
    I = k.instances()
    if len(I):
        K = str(k).split(".")[-1]
        #print(K)
        if K.startswith("Blueprint"):
            for i in I:
                destroy_entity(i)
            destroy_entity(k)
            
ontoB.save(file = "../kg/new/pbn_t3_5_noBP.owl")
debug(ontoB)
ontoB.destroy()


# Overview

*  _PBNThing_  --  19186 instances.
*  _BenefReturn_  --  1606 instances.
*  _Benef_  --  1409 instances.
*  _Article_  --  376 instances.
*  _Risk_  --  3418 instances.
*  _ISO_Scale_  --  5 instances.
*  _RiskHealth_  --  14 instances.
*  _RiskType_  --  22 instances.
*  _Stakeholder_  --  2640 instances.
*  _Stakeholder_Type_  --  3 instances.
*  _Technology_  --  3915 instances.
*  _ISO_Impact_  --  24 instances.
*  _ISO_Purpose_  --  18 instances.
*  _Mitigation_  --  5295 instances.
*  _BP_Enabler_  --  33 instances.
*  _BP_Transmission_  --  12 instances.
*  _BP_Scale_  --  3 instances.
*  _BP_Phase_  --  4 instances.
*  _BP_Permanent_  --  4 instances.
*  _BP_Type_  --  2 instances.
*  _BP_Intervention_  --  40 instances.
*  _PBNCategory_  --  330 instances.
*  _BP_Theme_  --  10 instances.
*  _BP_Category_  --  3 instances.


# Keeping only PROBONO Blueprints

In [6]:
ontoC = get_ontology("../kg/new/pbn_t3_5.owl").load()
ontoC.metadata.comment[4] = 'Version: 0.3.b'
ontoC.metadata.comment[5] = 'VersionComment: 0.3 - but only literature review (and automated blueprints)'

debug(ontoC)
with ontoC:
    for k in ontoC.classes():
        I = k.instances()
        if len(I):
            K = str(k).split(".")[-1]
            if not( (K.startswith("Blueprint") or K.startswith("BP_") or K == "PBNThing" ) ):
                print("Destroy",K)
                for i in I:
                    destroy_entity(i)
                destroy_entity(k)
            else:
                print("Keep",K)

debug(ontoC)
ontoC.save(file = "../kg/new/pbn_t3_5_BP.owl")
ontoC.destroy()


# Overview

*  _BP_Enabler_  --  33 instances.
*  _BP_Transmission_  --  12 instances.
*  _Blueprint_  --  74 instances.
*  _BP_Scale_  --  3 instances.
*  _BP_Phase_  --  4 instances.
*  _BP_Permanent_  --  4 instances.
*  _BP_Type_  --  2 instances.
*  _BP_Intervention_  --  40 instances.
*  _BP_Theme_  --  10 instances.
*  _BP_Category_  --  3 instances.
*  _BenefReturn_  --  1606 instances.
*  _Benef_  --  1409 instances.
*  _Article_  --  376 instances.
*  _Risk_  --  3418 instances.
*  _ISO_Scale_  --  5 instances.
*  _RiskHealth_  --  14 instances.
*  _RiskType_  --  22 instances.
*  _Stakeholder_  --  2640 instances.
*  _Stakeholder_Type_  --  3 instances.
*  _Technology_  --  3915 instances.
*  _ISO_Impact_  --  24 instances.
*  _ISO_Purpose_  --  18 instances.
*  _Mitigation_  --  5295 instances.
*  _PBNCategory_  --  330 instances.
*  _aBlueprint_  --  50 instances.
*  _PBNThing_  --  19260 instances.
Keep BP_Enabler
Keep BP_Transmission
Keep Blueprint
Keep BP_Scale
Keep BP

# Checks

In [7]:
ontoCheck = get_ontology("../kg/new/pbn_t3_5_BP.owl").load()
debug(ontoCheck)
ontoCheck.destroy()


# Overview

*  _BP_Enabler_  --  33 instances.
*  _BP_Transmission_  --  12 instances.
*  _Blueprint_  --  24 instances.
*  _BP_Scale_  --  3 instances.
*  _BP_Phase_  --  4 instances.
*  _BP_Permanent_  --  4 instances.
*  _BP_Type_  --  2 instances.
*  _BP_Intervention_  --  40 instances.
*  _BP_Theme_  --  10 instances.
*  _BP_Category_  --  3 instances.
*  _PBNThing_  --  135 instances.


In [17]:
if 0:
    OA = get_ontology("../kg/new/pbn_t3_5_BP.owl").load()
    OB = get_ontology("../kg/new/pbn_t3_5_noBP.owl").load()

    OA.mergeonto(OB, search=True, deleteduplicates=True)

NameError: name 'getontology' is not defined

# Checks

In [3]:
if 0:
    onto = get_ontology("../kg/pbn_t3_5.owl").load()
    onto.metadata.comment[4] = 'Version: 0.3.c'
    onto.metadata.comment[5] = 'VersionComment: 0.3 - total, reasoned'
    debug(onto)
    # Destroy empty classes
    for k in onto.classes():
        if not len(k.instances()):
            destroy_entity(k)
    # Destroy relations
    for k in onto.properties():
        N = len([x for x in k.get_relations()])
        if not N:
            destroy_entity(k)
    # Finetune world reasoning
    print("Start",NOW())
    with onto:
        sync_reasoner_pellet(infer_property_values = True, infer_data_property_values = True)
    print("Stop",NOW())
    # Saving the KG
    onto.save(file = "../kg/new/pbn_t3_5_reasoned.owl")
    onto.destroy()
    del onto
